In [11]:
from tkinter import messagebox
import numpy as np

import random
import base64

from src.helper.file import File
from src.helper.cipher import encrypt_vigenere

import module as md
import extractor as ex

from datetime import timedelta
import cv2
import numpy as np
import os, os.path

### ================= UNTUK EXTRACT GAMBAR DARI VIDEO ================

In [12]:
def get_saving_frames_durations(cap, saving_fps):
    """A function that returns the list of durations where to save the frames"""
    s = []
    # get the clip duration by dividing number of frames by the number of frames per second
    clip_duration = cap.get(cv2.CAP_PROP_FRAME_COUNT) / cap.get(cv2.CAP_PROP_FPS)
    # use np.arange() to make floating-point steps
    for i in np.arange(0, clip_duration, 1 / saving_fps):
        s.append(i)
    return s

In [13]:
width, height = 0,0
fps = 0
def main(video_file):
    global width
    global height
    global fps
    filename, _ = os.path.splitext(video_file)
    filename += "-opencv"
    # make a folder by the name of the video file
    if not os.path.isdir(filename):
        os.mkdir(filename)
    # read the video file    
    cap = cv2.VideoCapture(video_file)
    width = cap.get(cv2.CAP_PROP_FRAME_WIDTH )
    height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT )
    # get the FPS of the video
    fps = cap.get(cv2.CAP_PROP_FPS)
    fps = int(fps)
    # if the SAVING_FRAMES_PER_SECOND is above video FPS, then set it to FPS (as maximum)
    saving_frames_per_second = fps
    # get the list of duration spots to save
    saving_frames_durations = get_saving_frames_durations(cap, saving_frames_per_second)
    # start the loop
    count = 0
    while True:
        is_read, frame = cap.read()
        if not is_read:
            # break out of the loop if there are no frames to read
            break
        # get the duration by dividing the frame count by the FPS
        frame_duration = count / fps
        try:
            # get the earliest duration to save
            closest_duration = saving_frames_durations[0]
        except IndexError:
            # the list is empty, all duration frames were saved
            break
        if frame_duration >= closest_duration:
            # if closest duration is less than or equals the frame duration, 
            # then save the frame
            cv2.imwrite(os.path.join(filename, f"{count}.jpg"), frame) 
            # drop the duration spot from the list, since this duration spot is already saved
            try:
                saving_frames_durations.pop(0)
            except IndexError:
                pass
        # increment the frame count
        count += 1

### ================== MILIH FRAME ACAK LALU MASUKKIN PESAN PAKAI BPCS ===============

In [14]:
path = './sample/video/tes.mp4'
main(path)

In [15]:
DIR = './sample/video/tes-opencv/'
frame_count = (len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))]))

In [16]:
frame = random.randint(0, frame_count)
file = './sample/video/tes-opencv/'+str(frame)+'.jpg'
message = 'short.txt'
key = 'anu'

insert = md.Inserter(file, message, key)

inserted_image = insert.insert_message(encrypted=True, randomize=False, method='bpcs', alpha=0.3)

output = './sample/video/tes-opencv/'+str(frame)+'.jpg'
output_file = File(output)
output_file.write_image_file(inserted_image)
print(frame)

9


### ================ GABUNGIN VIDEO DENGAN GAMBAR YANG ADA PESANNYA ==============

In [8]:
 # choose codec according to format needed
fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
video = cv2.VideoWriter('video.mp4', fourcc, frameSize = (int(width), int(height)), fps=int(fps))

for j in range(0,59):
    path = './sample/video/tes-opencv/' + str(j) + '.jpg'
    img = cv2.imread('./sample/video/tes-opencv/' + str(j) + '.jpg')
    video.write(img)

cv2.destroyAllWindows()
video.release()

### ===========================EXTRACTOR PESAN TERSEMBUNYI===============================

In [17]:
path = 'video.mp4'
main(path)

In [ ]:
file = 'video.mp4'
key = 'anu'

extract = ex.Extractor (file, key)
extract.extract_message()
extract.parse_message()

output = 'short_output.txt'
output_file = File (output)
byte = extract.write_secret_message()
output_file.write_files(byte)